In [1]:

import os
import pickle

from scipy.stats import entropy
import tqdm

import numpy as np
import pandas as pd


In [2]:


def abrirBaseSerie(caracteristica, diretorioCaracteristicas) -> dict:
    arquivoPickle = f'{diretorioCaracteristicas}{caracteristica}_serieTemporal.pickle'
    
    try:
        with open(arquivoPickle, 'rb') as f:
            return pickle.load(f, fix_imports=False)
        
    except FileNotFoundError:
        return {}
            
    except Exception as e:
        print(f"Arquivo Pickle com Erro \n{e}")
        return {}

    
def parseMediaPonderada(series) -> float:
    try:
        series_media = series.mean(skipna=True)
        return series_media
    except Exception:
        return series


def parseVariancia(series) -> float:
    try:
        series_var = series.var(ddof=0, skipna=True)
        return series_var
    except Exception:
        series_media = parseMediaPonderada(series)
        return series_media

    
def parseMediaMovelPonderada(series) -> float:
    try:
        if len(series) > 7:
            convolucao = np.convolve(series, np.ones(7), 'valid')
            mediaMovel = sum(convolucao) / (len(series) - 7)
            return mediaMovel
        else:
            series_media = parseMediaPonderada(series)
            return series_media
        
    except Exception:
        series_media = parseMediaPonderada(series)
        return series_media


def parseEntropia(series) -> float:
    try:
        series_entropy = entropy(series.value_counts())
        return series_entropy
    except Exception:
        series_media = parseMediaPonderada(series)
        return series_media

    
atributo_volume_tweets, atributo_volume_tweets_noite = 'volumeTweets', 'volumeTweetsNoite'
atributo_indice_insonia, atributo_pronome_1_pessoa = 'indiceInsonia', 'pronome1Pessoa'
atributo_pronome_2_pessoa, atributo_pronome_3_pessoa = 'pronome2Pessoa', 'pronome3Pessoa'
atributo_valencia, atributo_ativacao, atributo_termos_depressivos = 'valencia', 'ativacao', 'termosDepressivos'
atributo_medicamentos_anti_depressivo, atributo_grafo_social = 'medicamentosAntiDepressivo', 'grafoSocial'
atributo_caracteres_orientais, atributo_emojis = 'caracteresOrientais', 'emojis'
atributo_links, atributo_midia, atributo_curtidas = 'links', 'midia', 'curtidas'
atributo_hashtags, atributo_retweets = 'hashtags', 'retweets'
atributo_mencoes, atributo_polaridade, atributo_subjetividade = 'mencoes', 'polaridade', 'subjetividade'

listaAtributos = [atributo_volume_tweets,
                  atributo_indice_insonia, atributo_pronome_1_pessoa,
                  atributo_pronome_2_pessoa, atributo_pronome_3_pessoa,
                  atributo_valencia, atributo_ativacao,
                  atributo_termos_depressivos, atributo_grafo_social,
                  atributo_medicamentos_anti_depressivo,
                  atributo_caracteres_orientais, atributo_emojis,
                  atributo_links, atributo_midia, atributo_curtidas,
                  atributo_hashtags, atributo_retweets, atributo_mencoes,
                  atributo_polaridade, atributo_subjetividade]


In [3]:

periodo = 'pandemia'
grupo = 'depressao'
diretorioCaracteristicas = f'datasets/{periodo}/caracteristicas/{grupo}/'

arquivoVetoresCaracteristicas = f'{diretorioCaracteristicas}{periodo}_{grupo}_vetoresCaracteristicas.csv'

listaCaracteristicas = [[f'{i}_media', f'{i}_variancia', f'{i}_mediaMovelPonterada', f'{i}_entropia'] for i in listaAtributos]
listaCaracteristicas = [i for lista in listaCaracteristicas for i in lista]

listaCandidatos = list(set(abrirBaseSerie(atributo_volume_tweets, diretorioCaracteristicas)))
listaCandidatos.sort()

if os.path.isfile(arquivoVetoresCaracteristicas):
    df = pd.read_csv(arquivoVetoresCaracteristicas, sep=';', index_col=0, encoding='utf-8')
    listaCandidatosProcessados = set(df.index)
else:
    df = pd.DataFrame(columns=listaCaracteristicas, index=listaCandidatos, data=0.0)
    listaCandidatosProcessados = set()

listaSeriesTemporais = [fnames for fnames in os.listdir(diretorioCaracteristicas) if '_serieTemporal' in fnames]

listaNovosCandidatos = list(set(listaCandidatos) - set(listaCandidatosProcessados))
listaNovosCandidatos.sort()

del listaCandidatos
del listaCandidatosProcessados

if listaNovosCandidatos:
    
    for candidato in listaNovosCandidatos:
        df.loc[candidato] = 0.0
        
    df.sort_index(axis=0, inplace=True)
    
    pbar = tqdm.tqdm(listaSeriesTemporais, colour="green")
        
    # carregando series temporais de cada caracteristicas
    for serie in pbar:
        seriesTemporais = {}
        atributo = serie.rsplit('_', 1)[0]
        pbar.set_description(f"Sumarizando atributos {periodo} {grupo} {atributo}")
        
        with open(diretorioCaracteristicas + serie, 'rb') as f:
            seriesTemporais[atributo] = pickle.load(f)
        
        for candidato in listaNovosCandidatos:
            
            for atributo in seriesTemporais.keys():
                df.loc[candidato][f'{atributo}_media'] = parseMediaPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_variancia'] = parseVariancia(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_mediaMovelPonterada'] = parseMediaMovelPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_entropia'] = parseEntropia(seriesTemporais[atributo][candidato])
                    
    if not os.path.exists(arquivoVetoresCaracteristicas):
        df.to_csv(arquivoVetoresCaracteristicas, sep=';', encoding='utf-8')
    #else:
        #df.to_csv(arquivoVetoresCaracteristicas, sep=';', header=False, mode='a', encoding='utf-8')


Sumarizando atributos pandemia depressao volumeTweets: 100%|██████████| 20/20 [06:21<00:00, 19.08s/it]             


In [4]:

periodo = 'pandemia'
grupo = 'controle'
diretorioCaracteristicas = f'datasets/{periodo}/caracteristicas/{grupo}/'

arquivoVetoresCaracteristicas = f'{diretorioCaracteristicas}{periodo}_{grupo}_vetoresCaracteristicas.csv'

listaCaracteristicas = [[f'{i}_media', f'{i}_variancia', f'{i}_mediaMovelPonterada', f'{i}_entropia'] for i in listaAtributos]
listaCaracteristicas = [i for lista in listaCaracteristicas for i in lista]

listaCandidatos = list(set(abrirBaseSerie(atributo_volume_tweets, diretorioCaracteristicas)))
listaCandidatos.sort()

if os.path.isfile(arquivoVetoresCaracteristicas):
    df = pd.read_csv(arquivoVetoresCaracteristicas, sep=';', index_col=0, encoding='utf-8')
    listaCandidatosProcessados = set(df.index)
else:
    df = pd.DataFrame(columns=listaCaracteristicas, index=listaCandidatos, data=0.0)
    listaCandidatosProcessados = set()

listaSeriesTemporais = [fnames for fnames in os.listdir(diretorioCaracteristicas) if '_serieTemporal' in fnames]

listaNovosCandidatos = list(set(listaCandidatos) - set(listaCandidatosProcessados))
listaNovosCandidatos.sort()

del listaCandidatos
del listaCandidatosProcessados

if listaNovosCandidatos:
    
    for candidato in listaNovosCandidatos:
        df.loc[candidato] = 0.0
        
    df.sort_index(axis=0, inplace=True)
    
    pbar = tqdm.tqdm(listaSeriesTemporais, colour="green")
        
    # carregando series temporais de cada caracteristicas
    for serie in pbar:
        seriesTemporais = {}
        atributo = serie.rsplit('_', 1)[0]
        pbar.set_description(f"Sumarizando atributos {periodo} {grupo} {atributo}")
        
        with open(diretorioCaracteristicas + serie, 'rb') as f:
            seriesTemporais[atributo] = pickle.load(f)
        
        for candidato in listaNovosCandidatos:
            
            for atributo in seriesTemporais.keys():
                df.loc[candidato][f'{atributo}_media'] = parseMediaPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_variancia'] = parseVariancia(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_mediaMovelPonterada'] = parseMediaMovelPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_entropia'] = parseEntropia(seriesTemporais[atributo][candidato])
                    
    if not os.path.exists(arquivoVetoresCaracteristicas):
        df.to_csv(arquivoVetoresCaracteristicas, sep=';', encoding='utf-8')
    #else:
        #df.to_csv(arquivoVetoresCaracteristicas, sep=';', header=False, mode='a', encoding='utf-8')

Sumarizando atributos pandemia controle volumeTweets: 100%|██████████| 20/20 [12:06<00:00, 36.30s/it]             


In [5]:

periodo = 'pre-pandemia'
grupo = 'depressao'
diretorioCaracteristicas = f'datasets/{periodo}/caracteristicas/{grupo}/'

arquivoVetoresCaracteristicas = f'{diretorioCaracteristicas}{periodo}_{grupo}_vetoresCaracteristicas.csv'

listaCaracteristicas = [[f'{i}_media', f'{i}_variancia', f'{i}_mediaMovelPonterada', f'{i}_entropia'] for i in listaAtributos]
listaCaracteristicas = [i for lista in listaCaracteristicas for i in lista]

listaCandidatos = list(set(abrirBaseSerie(atributo_volume_tweets, diretorioCaracteristicas)))
listaCandidatos.sort()

if os.path.isfile(arquivoVetoresCaracteristicas):
    df = pd.read_csv(arquivoVetoresCaracteristicas, sep=';', index_col=0, encoding='utf-8')
    listaCandidatosProcessados = set(df.index)
else:
    df = pd.DataFrame(columns=listaCaracteristicas, index=listaCandidatos, data=0.0)
    listaCandidatosProcessados = set()

listaSeriesTemporais = [fnames for fnames in os.listdir(diretorioCaracteristicas) if '_serieTemporal' in fnames]

listaNovosCandidatos = list(set(listaCandidatos) - set(listaCandidatosProcessados))
listaNovosCandidatos.sort()

del listaCandidatos
del listaCandidatosProcessados

if listaNovosCandidatos:
    
    for candidato in listaNovosCandidatos:
        df.loc[candidato] = 0.0
        
    df.sort_index(axis=0, inplace=True)
    
    pbar = tqdm.tqdm(listaSeriesTemporais, colour="green")
        
    # carregando series temporais de cada caracteristicas
    for serie in pbar:
        seriesTemporais = {}
        atributo = serie.rsplit('_', 1)[0]
        pbar.set_description(f"Sumarizando atributos {periodo} {grupo} {atributo}")
        
        with open(diretorioCaracteristicas + serie, 'rb') as f:
            seriesTemporais[atributo] = pickle.load(f)
        
        for candidato in listaNovosCandidatos:
            
            for atributo in seriesTemporais.keys():
                df.loc[candidato][f'{atributo}_media'] = parseMediaPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_variancia'] = parseVariancia(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_mediaMovelPonterada'] = parseMediaMovelPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_entropia'] = parseEntropia(seriesTemporais[atributo][candidato])
                    
    if not os.path.exists(arquivoVetoresCaracteristicas):
        df.to_csv(arquivoVetoresCaracteristicas, sep=';', encoding='utf-8')
    #else:
        #df.to_csv(arquivoVetoresCaracteristicas, sep=';', header=False, mode='a', encoding='utf-8')

Sumarizando atributos pre-pandemia depressao volumeTweets: 100%|██████████| 20/20 [06:22<00:00, 19.11s/it]             


In [6]:

periodo = 'pre-pandemia'
grupo = 'controle'
diretorioCaracteristicas = f'datasets/{periodo}/caracteristicas/{grupo}/'

arquivoVetoresCaracteristicas = f'{diretorioCaracteristicas}{periodo}_{grupo}_vetoresCaracteristicas.csv'

listaCaracteristicas = [[f'{i}_media', f'{i}_variancia', f'{i}_mediaMovelPonterada', f'{i}_entropia'] for i in listaAtributos]
listaCaracteristicas = [i for lista in listaCaracteristicas for i in lista]

listaCandidatos = list(set(abrirBaseSerie(atributo_volume_tweets, diretorioCaracteristicas)))
listaCandidatos.sort()

if os.path.isfile(arquivoVetoresCaracteristicas):
    df = pd.read_csv(arquivoVetoresCaracteristicas, sep=';', index_col=0, encoding='utf-8')
    listaCandidatosProcessados = set(df.index)
else:
    df = pd.DataFrame(columns=listaCaracteristicas, index=listaCandidatos, data=0.0)
    listaCandidatosProcessados = set()

listaSeriesTemporais = [fnames for fnames in os.listdir(diretorioCaracteristicas) if '_serieTemporal' in fnames]

listaNovosCandidatos = list(set(listaCandidatos) - set(listaCandidatosProcessados))
listaNovosCandidatos.sort()

del listaCandidatos
del listaCandidatosProcessados

if listaNovosCandidatos:
    
    for candidato in listaNovosCandidatos:
        df.loc[candidato] = 0.0
        
    df.sort_index(axis=0, inplace=True)
    
    pbar = tqdm.tqdm(listaSeriesTemporais, colour="green")
        
    # carregando series temporais de cada caracteristicas
    for serie in pbar:
        seriesTemporais = {}
        atributo = serie.rsplit('_', 1)[0]
        pbar.set_description(f"Sumarizando atributos {periodo} {grupo} {atributo}")
        
        with open(diretorioCaracteristicas + serie, 'rb') as f:
            seriesTemporais[atributo] = pickle.load(f)
        
        for candidato in listaNovosCandidatos:
            
            for atributo in seriesTemporais.keys():
                df.loc[candidato][f'{atributo}_media'] = parseMediaPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_variancia'] = parseVariancia(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_mediaMovelPonterada'] = parseMediaMovelPonderada(seriesTemporais[atributo][candidato])
                df.loc[candidato][f'{atributo}_entropia'] = parseEntropia(seriesTemporais[atributo][candidato])
                    
    if not os.path.exists(arquivoVetoresCaracteristicas):
        df.to_csv(arquivoVetoresCaracteristicas, sep=';', encoding='utf-8')
    #else:
        #df.to_csv(arquivoVetoresCaracteristicas, sep=';', header=False, mode='a', encoding='utf-8')

Sumarizando atributos pre-pandemia controle volumeTweets: 100%|██████████| 20/20 [05:37<00:00, 16.87s/it]             


In [6]:
    
def normalizarBaseDados(df, periodo):
    """_summary_

    Args:
        df (pd.DataFrame): _description_
        periodo (str): _description_
    """
    zscore = StandardScaler()
    data_zscore = zscore.fit_transform(df.iloc[:,:-1])
    df_zscore = pd.DataFrame(data=data_zscore)
    df_zscore['classe'] = df['classe']
    df_zscore.columns = df.columns
    
    df_zscore.to_csv(f'datasets/twitterbase_{periodo}_zscore.csv', sep=';', index=False)
    print(f'datasets/twitterbase_{periodo}_zscore.csv criada')
    
    df_minmax = MinMaxScaler()
    data_minmax = df_minmax.fit_transform(df.iloc[:,:-1])
    df_minmax = pd.DataFrame(data=data_minmax)
    df_minmax['classe'] = df['classe']
    df_minmax.columns = df.columns
    
    df_minmax.to_csv(f'datasets/twitterbase_{periodo}_minmax.csv', sep=';', index=False)
    print(f'datasets/twitterbase_{periodo}_minmax.csv criada')

def construirBaseDadosTwitter(periodo, diretorioDepressao, diretorioControle):
    """_summary_

    Args:
        periodo (str): _description_
        diretorioDepressao (str): _description_
        diretorioControle (str): _description_
    """
    dfDepressao = pd.read_csv(f'{diretorioDepressao}{periodo}_depressao_vetoresCaracteristicas.csv', sep=';', index_col=0)
    dfDepressao['classe'] = ['depressao'] * len(dfDepressao)
    
    dfControle = pd.read_csv(f'{diretorioControle}{periodo}_controle_vetoresCaracteristicas.csv', sep=';', index_col=0)
    dfControle['classe'] = ['controle'] * len(dfControle)
    
    df = pd.concat([dfDepressao, dfControle])
    df.fillna(0, inplace=True)
    df.sort_index(inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.to_csv(f'datasets/twitterbase_{periodo}.csv', sep=';', index=False)
    print(f'Dataset twitterbase_{periodo}.csv criada')
    
    #normalizarBaseDados(df, periodo)



In [8]:

periodo = 'pre-pandemia'

diretorioDepressao = f'datasets/{periodo}/caracteristicas/depressao/'
diretorioControle = f'datasets/{periodo}/caracteristicas/controle/'

construirBaseDadosTwitter(periodo, diretorioDepressao, diretorioControle)

Dataset twitterbase_pre-pandemia.csv criada


In [7]:
periodo = 'pandemia'

diretorioDepressao = f'datasets/{periodo}/caracteristicas/depressao/'
diretorioControle = f'datasets/{periodo}/caracteristicas/controle/'

construirBaseDadosTwitter(periodo, diretorioDepressao, diretorioControle)

Dataset twitterbase_pandemia.csv criada
